# Test nat_zacros package

## Import packages and get path to Zacros calculations

In [1]:
import os
import platform 
from pathlib import Path
from nat_zacros import lattice, state
import numpy as np

# Dictionary mapping usernames to data paths
user_paths = {
    'a-DJA'  : Path('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations'),
    'akandra': Path('/home/akandra/O_Pt111/zacros_calculations'),
    # Add more collaborators here as needed
    # note that file names and paths are case-sensitive here, even on Windows because file existence is checked in python
    # in a case sensitive manner
}

if platform.system() == 'Windows':
    username = os.getenv('USERNAME')
else:  # Linux/Mac
    username = os.getenv('USER')

# Get path for current user
if username in user_paths:
    userpath = user_paths[username]
else:
    raise ValueError(f"Unknown user: {username}. Please add your username and path to user_paths dictionary.")

print(f"Operating System : {platform.system()}")
print(f"username         : {username}")
print(f"userpath         : {userpath.as_posix()}")
# Show the type and value
print(f"repr(userpath)   : {repr(userpath)}")

Operating System : Windows
username         : a-DJA
userpath         : c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations
repr(userpath)   : WindowsPath('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations')


In [2]:
lat = lattice(userpath / 'fn_3leed/jobs/1/traj_1')
lat.site_coordinations
print( f'len(lat) = {len(lat)}' )

len(lat) = 1600


In [3]:
# create state object with empty occupation
ads = state(len(lat))
print(f'occupation empty lattice      = {repr(ads.occupation)} with '
      f'{np.count_nonzero(ads.occupation):3d} nonzero elements')
# load occupation from file
ads.get_state(userpath / 'fn_3leed/jobs/1/traj_1', idx=0)
print(f'occupation after get_lattice  = {repr(ads.occupation)} with '
      f'{np.count_nonzero(ads.occupation):3d} nonzero elements' )

occupation empty lattice      = array([0, 0, 0, ..., 0, 0, 0], shape=(1600,)) with   0 nonzero elements
occupation after get_lattice  = array([0, 0, 0, ..., 0, 0, 0], shape=(1600,)) with 176 nonzero elements


## Test new state methods

In [4]:
# Test coverage and count methods
print(f"State representation: {repr(ads)}")
print(f"\nCoverage: {ads.get_coverage():.4f}")
print(f"Number of adsorbates: {ads.get_n_adsorbates()}")
print(f"Total sites: {ads.nsites}")

State representation: state(nsites=1600, n_adsorbates=176, coverage=0.110)

Coverage: 0.1100
Number of adsorbates: 176
Total sites: 1600


In [5]:
# Load a different state and compare
ads2 = state(len(lat))
ads2.get_state(userpath / 'fn_3leed/jobs/1/traj_1', idx=10)  # Load state at idx=10

print(f"\nState at idx=0:  {repr(ads)}")
print(f"State at idx=10: {repr(ads2)}")
print(f"\nCoverage change: {ads2.get_coverage() - ads.get_coverage():.4f}")


State at idx=0:  state(nsites=1600, n_adsorbates=176, coverage=0.110)
State at idx=10: state(nsites=1600, n_adsorbates=176, coverage=0.110)

Coverage change: 0.0000


In [6]:
# Test coordinate extraction (composition: state USES lattice)
occupied_coords = ads.get_occupied_coords(lat)

print(f"\nOccupied coordinates shape: {occupied_coords.shape}")
print(f"First 5 occupied coordinates:\n{occupied_coords[:5]}")


Occupied coordinates shape: (176, 2)
First 5 occupied coordinates:
[[ 7.0528375  10.58708984]
 [11.28454    17.91661357]
 [14.105675   22.80296272]
 [16.92681    27.68931188]
 [21.1585125  35.01883561]]


In [7]:
# Test site query methods
occupied_sites = ads.get_occupied_sites()
empty_sites = ads.get_empty_sites()

print(f"\nOccupied sites (first 10): {occupied_sites[:10]}")
print(f"Number of occupied sites: {len(occupied_sites)}")
print(f"Number of empty sites: {len(empty_sites)}")
print(f"Total check: {len(occupied_sites) + len(empty_sites)} == {ads.nsites}")


Occupied sites (first 10): [ 4  7  9 11 14 35 46 59 72 73]
Number of occupied sites: 176
Number of empty sites: 1424
Total check: 1600 == 1600
